# **Creating pre-final dataset**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

### **Loading of all data**

In [52]:
data_Honza = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW.csv', encoding='utf-8', low_memory=False)
data_Honza_wkend = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW_WKEND.csv', encoding='utf-8', low_memory=False)
data_keboola = pd.read_csv('out.c-cleaning.ALL_POSITIONS_NEW.csv', encoding='utf-8', low_memory=False)
data_keboola_wkend = pd.read_csv('out.clean_VIKEND_END.csv', encoding='utf-8', low_memory=False)

data = pd.concat([data_Honza,data_Honza_wkend,data_keboola,data_keboola_wkend], ignore_index = True)
data = data.dropna(axis=0,subset=['DELAY'],how='any')

### **Adding columns for route name and day of week** 

In [53]:
data['ROUTE_NAME'] = data['TRIP_ID'].str[:3]

In [54]:
data['DATE'] = data['DATE'] + " 2020"

In [55]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [56]:
data['DAY_OF_WEEK'] = data['DATE'].dt.day_name()

In [57]:
data['DAY_OF_WEEK_NUM'] = data['DATE'].dt.dayofweek

### **We want route 331,333 +  in the morning traffic + towards the subway station**

In [58]:
morn_331_333 = data.loc[(data['HEADSIGN'] == 'Praha,Kačerov') 
                     & (data['START_TIME'] < '12:00:00') 
                     & ((data['ROUTE_NAME'] == '333') | (data['ROUTE_NAME'] == '331'))
                     , ['UNICORN', 'TRIP_ID', 'START_TIME', 'HEADSIGN', 'DATE',
                        'DELAY', 'DELAY_STOP_ARRIVAL', 'DELAY_STOP_DEPARTURE',
                        'ORIGIN_TIMESTAMP', 'SHAPE_DIST_TRAVELED', 'SPEED', 'LATITUDE',
                        'LONGITUDE','ROUTE_NAME','DAY_OF_WEEK']]

#morn_331_333.head()

In [59]:
# count of unique trip_id
len(morn_331_333['TRIP_ID'].unique())

27

In [60]:
# count of uniques unicorns z df data (allpositions)
unicorn_list = morn_331_333['UNICORN'].unique().tolist()
len(unicorn_list)

565

### **Dataframe with timetable schedule for all uniques trip_id (jizdni_rad.ipynb)**

In [61]:
info_stops_clean = pd.read_csv('timetable.csv', encoding='utf-8', low_memory=False)

In [62]:
info_stops_clean

,ARRIVAL_TIME,DEPARTURE_TIME,SHAPE_DIST_TRAVELED,STOP_ID,STOP_SEQUENCE,TRIP_ID,GEOMETRY_COORDINATES,STOP_NAME
0,9:31:00,9:31:00,0.00000,U2109Z2,1,331_243_200302,"[14.49348, 49.89506]","Jílové u Prahy,Náměstí"
1,9:33:00,9:33:00,0.65952,U2100Z2,2,331_243_200302,"[14.48552, 49.8974]","Jílové u Prahy,Učiliště"
2,9:37:00,9:37:00,2.37567,U1586Z2,3,331_243_200302,"[14.46857, 49.89977]","Jílové u Prahy,Kamenná Vrata"
3,9:39:00,9:39:00,4.03046,U1521Z2,4,331_243_200302,"[14.45916, 49.90944]","Okrouhlo,Zahořany"
4,9:43:00,9:43:00,5.86913,U1520Z2,5,331_243_200302,"[14.44766, 49.91885]",Okrouhlo
...,...,...,...,...,...,...,...,...
608,8:15:00,8:15:00,16.22898,U749Z2,24,333_1012_200307,"[14.45197, 50.02019]",Tempo
609,8:16:00,8:16:00,16.56781,U656Z2,25,333_1012_200307,"[14.45149, 50.02318]",Sídliště Krč
610,8:17:00,8:17:00,17.03180,U893Z4,26,333_1012_200307,"[14.45143, 50.02735]",Zálesí
611,8:19:00,8:19:00,17.58466,U488Z7,27,333_1012_200307,"[14.45515, 50.03038]",Nemocnice Krč


In [63]:
# selection of columns we want
stop_dist = info_stops_clean.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','STOP_ID','STOP_SEQUENCE','STOP_NAME','ARRIVAL_TIME']]
# stop_dist

## **Empty DF and appending all the data we want -> merging bus and stop info**

In [64]:
# new DF
final_tab = pd.DataFrame()

In [65]:
index = 0
final_tab = pd.DataFrame()

for uni in unicorn_list:
    #to see the progress
    print(round(index/len(unicorn_list)*100), '%')
    index+=1
    
    #smaller dataframe based on selected uni
    unicorn_find = morn_331_333[morn_331_333['UNICORN'] == uni]
    
    #columns I want from the bus (all positions) dataframe
    bus_dist = unicorn_find.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','DELAY','DATE','DAY_OF_WEEK',
                                   'START_TIME','HEADSIGN','ROUTE_NAME','ORIGIN_TIMESTAMP']]
    
    #these values are repeating for one unicorn so I want just the 1 to tell me the information
    gtfs_trip_id = bus_dist['TRIP_ID'].tolist()[0]
    trip_date = bus_dist['DATE'].tolist()[0]
    trip_day = bus_dist['DAY_OF_WEEK'].tolist()[0]
    trip_start_time = bus_dist['START_TIME'].tolist()[0]
    trip_headsign = bus_dist['HEADSIGN'].tolist()[0]
    route_name = bus_dist['ROUTE_NAME'].tolist()[0]    
    
    #for every bus stop in timetable for GTFS_TRIP_ID
    for i, stop in stop_dist[stop_dist['TRIP_ID'] == gtfs_trip_id].iterrows():
        
        #I am looking for the closest position(value), I am gonna set the first value as really far away
        min_distance_so_far = 100000
        #and delay to 0 
        stop_delay = 0
        
        for j, position in bus_dist.iterrows():
            
            # absolute value of the distance of stop - position
            actual_distance = abs(stop['SHAPE_DIST_TRAVELED'] - position['SHAPE_DIST_TRAVELED'])

            # if this value is closer than 100000 or closer than the last one found
            if (min_distance_so_far > actual_distance):
                #save it
                min_distance_so_far = actual_distance
                #with delay and origin timestamp
                stop_delay = position['DELAY']
                origin_timestamp = position['ORIGIN_TIMESTAMP'].split(' ')[4]
            
        # now for the bus stop I checked all the positions and return only the closest one
        #look of the final DF with columns and values I want
        df = pd.DataFrame(
            {
                "TRIP_ID": [gtfs_trip_id],
                "ROUTE" : [route_name],
                "DAY_OF_WEEK": [trip_day],
                "DATE": [trip_date],
                "UNICORN": [uni],
                "START_TIME" : [trip_start_time],
                "HEADSIGN" : [trip_headsign],
                "STOP_ID": [stop['STOP_ID']],
                "STOP_SEQUENCE": [stop['STOP_SEQUENCE']],
                "STOP_NAME" : [stop['STOP_NAME']],
                "SHAPE_DIST_TRAVELED": [stop['SHAPE_DIST_TRAVELED']],
                "DELAY": [stop_delay],
                "ORIGIN_TIMESTAMP": [origin_timestamp],
                "TIMETABLE_ARRIVAL_TIME":[stop['ARRIVAL_TIME']]
            }
        )
        # append to the big DF
        final_tab = final_tab.append(df, ignore_index = True)

0 %
0 %
0 %
1 %
1 %
1 %
1 %
1 %
1 %
2 %
2 %
2 %
2 %
2 %
2 %
3 %
3 %
3 %
3 %
3 %
4 %
4 %
4 %
4 %
4 %
4 %
5 %
5 %
5 %
5 %
5 %
5 %
6 %
6 %
6 %
6 %
6 %
7 %
7 %
7 %
7 %
7 %
7 %
8 %
8 %
8 %
8 %
8 %
8 %
9 %
9 %
9 %
9 %
9 %
10 %
10 %
10 %
10 %
10 %
10 %
11 %
11 %
11 %
11 %
11 %
12 %
12 %
12 %
12 %
12 %
12 %
13 %
13 %
13 %
13 %
13 %
13 %
14 %
14 %
14 %
14 %
14 %
15 %
15 %
15 %
15 %
15 %
15 %
16 %
16 %
16 %
16 %
16 %
16 %
17 %
17 %
17 %
17 %
17 %
18 %
18 %
18 %
18 %
18 %
18 %
19 %
19 %
19 %
19 %
19 %
19 %
20 %
20 %
20 %
20 %
20 %
21 %
21 %
21 %
21 %
21 %
21 %
22 %
22 %
22 %
22 %
22 %
22 %
23 %
23 %
23 %
23 %
23 %
24 %
24 %
24 %
24 %
24 %
24 %
25 %
25 %
25 %
25 %
25 %
25 %
26 %
26 %
26 %
26 %
26 %
27 %
27 %
27 %
27 %
27 %
27 %
28 %
28 %
28 %
28 %
28 %
28 %
29 %
29 %
29 %
29 %
29 %
30 %
30 %
30 %
30 %
30 %
30 %
31 %
31 %
31 %
31 %
31 %
32 %
32 %
32 %
32 %
32 %
32 %
33 %
33 %
33 %
33 %
33 %
33 %
34 %
34 %
34 %
34 %
34 %
35 %
35 %
35 %
35 %
35 %
35 %
36 %
36 %
36 %
36 %
36 %
36 %
37 %
37 %
37 %
37 %

In [66]:
#just check the count of trip_ids
len(final_tab['UNICORN'].unique())

565

In [67]:
#final_tab

In [68]:
final_tab['TRIP_ID'].unique()

array(['331_241_200302', '333_557_200302', '331_243_200302',
       '333_1064_200302', '333_715_200302', '333_1072_200302',
       '333_1052_200302', '331_260_200302', '333_677_200302',
       '333_751_200302', '333_1012_200302', '333_1094_200302',
       '333_1115_200302', '331_257_200302', '333_651_200307',
       '333_1026_200307', '333_653_200307', '333_671_200307',
       '333_1010_200307', '333_1037_200307', '333_636_200307',
       '333_1014_200307', '333_1047_200307', '333_655_200307',
       '333_1024_200307', '333_673_200307', '333_1012_200307'],
      dtype=object)

Unique trip_id --> unification of the number of bus stops -> STOP_SEQUENCE from Dolní Břežany,Náměstí

- 333_655_200307 >= 2
- 333_1012_200307 >= 14 
- 333_655_200307 Dolní Břežany,Náměstí
- 333_1047_200307(Zvole) >= 7 
- 333_636_200307 Dolní Břežany,Náměstí
- 333_1037_200307(Březová-Oleško,Oleško) >= 14 
- 333_1010_200307(Březová-Oleško,Oleško) >= 19   
- 333_653_200307 Dolní Břežany,Náměstí	
- 333_1026_200307(Zvole) >= 14
- 333_651_200307 Dolní Břežany,Náměstí	
- 333_1115_200302 (Zvole) >=12
- 333_1094_200302	(Březová-Oleško,Oleško) >= 14
- 333_1012_200302(Březová-Oleško,Oleško) >= 14
- 333_751_200302 (Dolní Břežany,Obecní úřad) >= 2
- 333_677_200302 (Dolní Břežany,Obecní úřad) >= 2
- 333_1052_200302(Zvole) >= 12
- 333_1072_200302(Zvole,Nová Zvole) >= 14
- 333_715_200302 (Dolní Břežany,Obecní úřad) >= 2
- 333_1064_200302 (Březová-Oleško,Oleško) >= 14
- 333_557_200302 (Dolní Břežany,Zálepy) >= 7
- 333_1014_200307 (Zvole) >= 14
- 333_671_200307 Dolní Břežany,Náměstí
- 333_1024_200307 (Zvole) >= 7
- 333_673_200307 Dolní Břežany,Náměstí

- 331_241_200302 (Jílové u Prahy,Náměstí) >= 14	
- 331_243_200302 (Jílové u Prahy,Náměstí) >= 14	
- 331_257_200302 (Okrouhlo,Zahořany) >= 11
- 331_260_200302 (Zvole) >= 8

In [69]:
# I will just choose the ones they have in common and they are closet to the city
final_clean =   final_tab[
((final_tab['TRIP_ID'] == '333_677_200302') & (final_tab['STOP_SEQUENCE'] >= 2)) 
| ((final_tab['TRIP_ID'] == '333_715_200302') & (final_tab['STOP_SEQUENCE'] >= 2))
| ((final_tab['TRIP_ID'] == '333_751_200302') & (final_tab['STOP_SEQUENCE'] >= 2))
| ((final_tab['TRIP_ID'] == '333_1012_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1047_200307') & (final_tab['STOP_SEQUENCE'] >= 7))
| ((final_tab['TRIP_ID'] == '333_1037_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1010_200307') & (final_tab['STOP_SEQUENCE'] >= 19))
| ((final_tab['TRIP_ID'] == '333_1026_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1115_200302') & (final_tab['STOP_SEQUENCE'] >= 12))
| ((final_tab['TRIP_ID'] == '333_1094_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1012_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1052_200302') & (final_tab['STOP_SEQUENCE'] >= 12))
| ((final_tab['TRIP_ID'] == '333_1072_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1064_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_557_200302') & (final_tab['STOP_SEQUENCE'] >= 7))
| ((final_tab['TRIP_ID'] == '331_241_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '331_243_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '331_257_200302') & (final_tab['STOP_SEQUENCE'] >= 11))
| ((final_tab['TRIP_ID'] == '331_260_200302') & (final_tab['STOP_SEQUENCE'] >= 8))
| (final_tab['TRIP_ID'] == '333_636_200307')
| (final_tab['TRIP_ID'] == '333_655_200307')
| (final_tab['TRIP_ID'] == '333_653_200307')
| (final_tab['TRIP_ID'] == '333_651_200307')
| (final_tab['TRIP_ID'] == '333_671_200307')
| ((final_tab['TRIP_ID'] == '333_1014_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1024_200307') & (final_tab['STOP_SEQUENCE'] >= 7))
| (final_tab['TRIP_ID'] == '333_673_200307')
]

final_clean

,TRIP_ID,ROUTE,DAY_OF_WEEK,DATE,UNICORN,START_TIME,HEADSIGN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY,ORIGIN_TIMESTAMP,TIMETABLE_ARRIVAL_TIME
13,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U1502Z2,14,"Dolní Břežany,Náměstí",13.17849,98.0,07:16:42,7:15:00
14,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U1501Z2,15,"Dolní Břežany,Na kopečku",13.80006,32.0,07:17:32,7:17:00
15,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U545Z2,16,Písnice,16.55734,45.0,07:21:40,7:21:00
16,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U546Z2,17,Ke Březině,16.93703,29.0,07:22:48,7:22:00
17,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U871Z2,18,Lipovická,17.13697,11.0,07:23:19,7:23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13101,333_671_200307,333,Saturday,2020-05-30,333_671_200307_30_May,11:30:00,"Praha,Kačerov",U749Z2,11,Tempo,6.84113,153.0,11:47:43,11:45:00
13102,333_671_200307,333,Saturday,2020-05-30,333_671_200307_30_May,11:30:00,"Praha,Kačerov",U656Z2,12,Sídliště Krč,7.17996,150.0,11:48:33,11:46:00
13103,333_671_200307,333,Saturday,2020-05-30,333_671_200307_30_May,11:30:00,"Praha,Kačerov",U893Z4,13,Zálesí,7.64396,167.0,11:49:41,11:47:00
13104,333_671_200307,333,Saturday,2020-05-30,333_671_200307_30_May,11:30:00,"Praha,Kačerov",U488Z7,14,Nemocnice Krč,8.19682,140.0,11:51:21,11:49:00


In [70]:
#final check
#final_clean[(final_clean['STOP_NAME'] == 'Dolní Břežany,Náměstí')]
len(final_clean.loc[:,'UNICORN'].unique())

565

In [71]:
final_clean.to_csv('stops_vs_positions_clean.csv', encoding="utf-8", index=False)